In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [ ]:
spark = SparkSession \
    .builder \
    .appName("Spark_movies_query") \
    .config("spark.master", "spark://ZHUYANZHAO-HWPC.localdomain:7077") \
    .config("spark.jars.packages", "mysql:mysql-connector-java:8.0.22") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.num", "1") \
    .getOrCreate()

In [ ]:
df_movies = spark.read.format("jdbc") \
    .options(driver = "com.mysql.cj.jdbc.Driver",
        url = "jdbc:mysql://localhost:3306/spark_movies",
        dbtable = "movies",
        user = "root",
        password = "zhuyz928"
        ).load()
df_movies_prod_companies =  spark.read.format("jdbc") \
    .options(driver = "com.mysql.cj.jdbc.Driver",
        url = "jdbc:mysql://localhost:3306/spark_movies",
        dbtable = "movie_prod_companies",
        user = "root",
        password = "zhuyz928"
        ).load()

In [ ]:
## First query: Get all movies produced by "Marvel Studios", and show the results as the Descending order by vote_average
df_movies_prod_companies = df_movies_prod_companies.withColumnRenamed('name', 'company_name')
df_movies_joined = df_movies_prod_companies.join(df_movies, df_movies.tmdb_id == df_movies_prod_companies.movie_tmdb_id,"left")
df_result = df_movies_joined.where(df_movies_joined.company_name == 'Marvel Studios') \
    .na.drop(subset=['vote_average']) \
    .sort(f.col('vote_average').desc())
df_result.show(df_result.count())

In [ ]:
## Second query: List the movie with the biggest profit ratio within each genre after 2000-01-01
df_movies_genres =  spark.read.format("jdbc") \
    .options(driver = "com.mysql.cj.jdbc.Driver",
        url = "jdbc:mysql://localhost:3306/spark_movies",
        dbtable = "movie_genres",
        user = "root",
        password = "zhuyz928"
        ).load()
df_movies = df_movies.withColumn('release_date', f.to_date(f.col('release_date')))
df_movies_filtered = df_movies.na.drop(subset=['budget', 'revenue']) \
    .filter((f.col('budget') > 0) & (f.col('revenue') > 0)) \
    .filter(f.col('revenue') >= f.col('budget')) \
    .filter(f.col('release_date') >= f.lit("2000-01-01"))
df_movies_filtered.count()

In [ ]:
df_movies_filtered = df_movies_filtered.withColumn('profit_ratio', f.round(f.col('revenue') / f.col('budget'), 2))
df_movies_genres = df_movies_genres.na.drop('any')
df_movies_joined = df_movies_genres.join(df_movies_filtered, df_movies_filtered.tmdb_id == df_movies_genres.movie_tmdb_id, "inner") \
    .withColumnRenamed('name', 'genre_name')
df_movies_joined.count()

In [ ]:
from pyspark.sql.window import Window
windowMovie = Window.partitionBy('genre_name').orderBy(f.col('profit_ratio').desc())
df_result = df_movies_joined.withColumn('row', f.row_number().over(windowMovie)) \
  .filter(f.col("row") == 1).drop("row") \
  .show()